# Using Mujoco simulator

On this page, I want to use a simulator to modelise the robot physics. 



In [1]:
import mujoco
import mujoco.viewer
import time

# Remplacez 'votre_modele.xml' par le chemin d'accès à votre fichier XML
try:
    m = mujoco.MjModel.from_xml_path('robot_models/h1_2_description/h1_2_handless.xml')
    d = mujoco.MjData(m)
except Exception as e:
    print(f"Erreur lors du chargement du fichier XML : {e}")
    exit()

# Lancer le visualiseur interactif
with mujoco.viewer.launch_passive(m, d) as viewer:
  # Boucle de simulation
  start = time.time()
  while viewer.is_running() and time.time() - start < 30:
    step_start = time.time()
    mujoco.mj_step(m, d)

    # Synchroniser le visualiseur
    with viewer.lock():
      viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = int(d.time > 0)
    viewer.sync()

    # Attendre pour respecter le temps de simulation
    time_until_next_step = m.opt.timestep - (time.time() - step_start)
    if time_until_next_step > 0:
      time.sleep(time_until_next_step)



/home/cpene/Documents/robot_playground/.venv/lib/python3.12/site-packages/glfw/__init__.py:917: GLFWError: (65548) b'Wayland: The platform does not provide the window position'
  warnings.warn(message, GLFWError)


In [27]:
# Display the robot on in initial position, with his feets close to the ground
q0 = pin.neutral(model)
q0[2] = 1.03

robot_visualizer.display(q0)

In [28]:
print("--- Preparing Dynamic Simulation ---")

# 1. Créer l'objet "data" associé au modèle. Indispensable.
data = model.createData()

# 2. Définir les conditions initiales
# On génère une configuration aléatoire au lieu d'une configuration neutre
q = pin.neutral(model)
q[2] = 1.03
q[7:] = np.random.uniform(model.lowerPositionLimit[7:], model.upperPositionLimit[7:])

v = np.zeros(model.nv)
tau = np.zeros(model.nv)
model.gravity.linear = np.array([0, 0, -9.81])
dt = 0.01  # Pas de temps de la simulation (10ms)

robot_visualizer.display(q)

for i in range(500): # On simule 500 pas, soit 5 secondes
        
    # CŒUR DE LA SIMULATION :
    # Calcule l'accélération (a) en fonction de l'état (q,v) et des efforts (tau)
    # C'est la "Forward Dynamics"
    a = pin.aba(model, data, q, v, tau)
    
    # On intègre l'accélération pour obtenir la nouvelle vitesse
    v_next = v + a * dt
    
    # On intègre la vitesse pour obtenir la nouvelle configuration
    # ATTENTION: On utilise pin.integrate, pas une simple addition (q + v*dt)!
    # C'est crucial pour gérer correctement les rotations.
    q_next = pin.integrate(model, q, v * dt)
    
    # On met à jour nos variables d'état
    q, v = q_next, v_next
    
    # On affiche le nouvel état du robot dans Meshcat
    robot_visualizer.display(q)
    
    # On attend 'dt' secondes pour que la simulation tourne en temps réel
    time.sleep(dt)

print("--- Simulation Finished ---")


--- Preparing Dynamic Simulation ---


NameError: name 'q_base_initiale' is not defined